In [1]:
# import modules for math and distributions
from math import exp
import numpy as np
from scipy.stats import gamma, norm, uniform, bernoulli
import pandas as pd
import statsmodels.api as sm
import rpy2
import patsy
import pdb

In [2]:
# required to be able to work with rpy2
%load_ext rpy2.ipython

In [3]:
# define inverse logit function
def logit_1(x): return(exp(x)/(1 + exp(x)))

In [4]:
# Initial set up following to HD2012
T = 40 # time periods
k = 5 # check-up times
theta = [-0.405, 0.0205, -0.00405]
gam = [-3, 0.05, -1.5, 0.1]

In [5]:
# make a function that does all of the above for an individual
def sim(T, k, gam, theta, patid=0):

    # define lists for holding A, L, U and Y
    A = -1*np.ones(T + 2) # A[-1] (last value) holds A in t = -1
    L = np.zeros(T+1)
    U = np.zeros(T+1)
    Y = -1*np.ones(T + 2)
    eps = np.zeros(T+1)
    lam = np.zeros(T+1) # prob of failure at each time period
    delta = np.zeros(T+1)

    # set the first value of U, U[0], to a 
    # randomly generated value from a uniform
    # distribution a measure of general health
    U[0] = uniform.rvs()
    eps[0] = norm.rvs(0, 20)
    L[0] = gamma.ppf(U[0], 3, scale=154) + eps[0]
    # L[0] = max(0, gamma.ppf(U[0], 3, scale=154) + eps[0])

    # set A[-1] to 0: held in last value of A
    A[-1] = 0
    Y[0] = 0
    
    # set A[0]
    A[0] = bernoulli.rvs(logit_1(theta[0] + theta[2] * (L[0] - 500)), size=1)
    
    if A[0] == 1:
        Ts = 0 
    else:
        Ts = -1
    
    lam[0] = logit_1(gam[0] + gam[2] * A[0])
    
    if lam[0] >= U[0]:
        Y[1] = 1
    else:
        Y[1] = 0
    # loop through each time period - stop when patient is dead or t = T + 1
    for t in range(1, T+1):
        if Y[t] == 0:
            delta[t] = norm.rvs(0, 0.05)
            U[t] = min(1, max(0, U[t-1] + delta[t]))
            if t % k != 0:
                L[t] = L[t-1]
                A[t] = A[t-1]
            else:
                eps[t] = norm.rvs(100 * (U[t] - 2), 50)
                L[t] = max(0, L[t-1] + 150 * A[t-k] * (1-A[t-k-1]) + eps[t])
                if A[t-1] == 0:
                    A[t] = bernoulli.rvs(logit_1(theta[0] + theta[1] * t + theta[2] * (L[t] - 500)), size=1)
                else:
                    A[t] = 1
                if A[t] == 1 and A[t-k] == 0: 
                    Ts = t
            ########################################
            # This is a check for debugging purposes
            # Comment it before the next push
            if Ts == -1:
                if A[t]:
                    print('There is an error...')
            ########################################
            lam[t] = logit_1(gam[0] + gam[1] * ((1 - A[t]) * t + A[t] * Ts) + gam[2] * A[t] + gam[3] * A[t] *(t - Ts))
            if (1 - np.prod(1 - lam)) >= U[0]:
                Y[t + 1] = 1
            else:
                Y[t+1] = 0
        else:
            break
    
    #we only need the data before death, so whatever value t is before the end of the
    #above loop - change this to numpy array and transpose.
    Y = np.ndarray.tolist(Y[1:(t+1)])
    U = np.ndarray.tolist(U[0:t])
    L = np.ndarray.tolist(L[0:t])
    A = np.ndarray.tolist(A[0:t])
    Ts = [Ts]*t

    df = np.vstack((Y, L, U, A, Ts))
    df = pd.DataFrame(df.T, columns=['Y', 'L', 'U', 'A', 'Ts'])
    df['Y'] = df['Y'].astype(int)
    df['A'] = df['A'].astype(int)
    df['patid'] = patid
    df.index.name = 'visit'
    return df.reset_index()

In [6]:
# use sim function to make a pandas DF for n patients
def get_sim_data(T, k, gam, theta, n = 1000):

    # get data for each of 1000 patients
    frames = [sim(T, k, gam, theta, patid=i) for i in range(n)]
    df = pd.concat(frames)
    # make new variables for the logit regression
    # including an intercept
    df["d1"] = (1-df['A'])*df['visit'] + df['A']*df['Ts']
    df["d3"] = df['A']*(df['visit']-df['Ts'])
    # df["time_Ts"] = df["visit"] - df["Ts"]
    df = df.set_index(['patid', 'visit'])
    df = df.sort_index()
    
    #create two new variables 
    df["L_100"] = df["L"]/100
    def func(x):
        x["Lav_100"] = x["L"].mean()/100
        return x

    df = df.groupby(level="patid").apply(func)
    
    # get the previous value of A, and set first value of A_1 per patient to zero.
    df['A_1'] = df.groupby(level="patid")['A'].shift(1)
    df['A_1'] = df['A_1'].fillna(0)

    return(df)

In [7]:
def get_weights(df):
    
    # only need data when A is not 1 yet
    df["As"] = df.groupby(level="patid")['A'].cumsum()
    df2 = df[df["As"] <= 1].copy(deep=True)

    # numerator
    f = "A ~ 1"
    y, X = patsy.dmatrices(f, df2.reset_index(), return_type = "dataframe")
    n_logit = sm.Logit(y, X, missing="raise")
    n_result = n_logit.fit(disp=0, maxiter=100)
    df2["pn"] = n_result.predict()
    
    # denominator
    f = "A ~ L"
    y, X = patsy.dmatrices(f, df2.reset_index(), return_type = "dataframe")
    d_logit = sm.Logit(y, X, missing="raise")
    d_result = d_logit.fit(disp=0, maxiter=100)
    df2["pd"] = d_result.predict()

    # if A == 0, change probabilities to 1 - prob
    df2['pn2'] = np.where(df2['A']==0, (1 - df2["pn"]), df2["pn"])
    df2['pd2'] = np.where(df2['A']==0, (1 - df2["pd"]), df2["pd"])

    # construct stabilized weights, don't forget to group by
    df2['cpn'] = df2.groupby(level=0)['pn2'].cumprod()
    df2['cpd'] = df2.groupby(level=0)['pd2'].cumprod()
    df2['sw'] = df2['cpn']/df2['cpd']

    #combine df and df2
    df["sw"] = np.nan
    df.loc[df2.index, "sw"] = df2["sw"]
    df["sw"] = df["sw"].fillna(method="pad")
    
    return(df)


In [8]:
df = get_sim_data(T, k, gam, theta, n = 1000)
df = get_weights(df)

In [9]:
tmp = df.reset_index()
tmp.head()

,patid,visit,Y,L,U,A,Ts,d1,d3,L_100,Lav_100,A_1,As,sw
0,0,0,1,68.341857,0.029688,0,-1.0,0.0,0.0,0.683419,0.683419,0.0,0,1.872436
1,1,0,0,277.215273,0.247495,0,5.0,0.0,-0.0,2.772153,2.167284,0.0,0,1.231702
2,1,1,0,277.215273,0.206139,0,5.0,1.0,-0.0,2.772153,2.167284,0.0,0,1.517090
3,1,2,0,277.215273,0.141244,0,5.0,2.0,-0.0,2.772153,2.167284,0.0,0,1.868603
4,1,3,0,277.215273,0.053809,0,5.0,3.0,-0.0,2.772153,2.167284,0.0,0,2.301561


In [15]:
%%R -i tmp -o co
library(ipw)
library(survey)
tmp$tstart <- tmp$visit - 1
co <- matrix(NA, nrow= 100, ncol = 4)
ipwsw <- ipwtm(exposure = A, family = "survival",
               numerator = ~ 1, denominator = ~ L,
               id = patid, timevar = visit, tstart = tstart,
               type = "first", data = tmp)
tmp$ipwsw <- ipwsw$ipw.weights
desipw <- svydesign(ids = ~ 1, data = tmp, weights = ~ ipwsw)
mdl <- svyglm(Y ~ d1 + A + d3, design = desipw, family = quasibinomial())
co <- coef(mdl)
print(summary(mdl))

SyntaxError: invalid syntax (<ipython-input-15-cc9ec025ca34>, line 5)

In [29]:
import rpy2.robjects as robjects
robjects.r('''
       runreg <- function(tmp){
       library(ipw)
       library(survey)
       tmp$tstart <- tmp$visit - 1
       ipwsw <- ipwtm(exposure = A, family = "survival", # survival model for weights
       numerator = ~ 1, denominator = ~ L,
       id = patid, timevar = visit, tstart = tstart,
       type = "first", data = tmp)
       tmp$ipwsw <- ipwsw$ipw.weights
       #desipw <- svydesign(ids = ~ 1, data = tmp, weights = ~ ipwsw)
       #mdl <- svyglm(Y ~ d1 + A + d3, design = desipw, family = quasibinomial())
       mdl <- glm(Y ~ d1 + A + d3, data = tmp, weights=ipwsw, family=binomial)
       return(coef(mdl))
       }
''')

r_runreg = robjects.globalenv['runreg']
r_runreg(tmp)

B = 100
co = np.zeros((B, 4))
for i in range(B):
    df = get_sim_data(T, k, gam, theta, n = 1000)
    df = get_weights(df)
    tmp = df.reset_index()
    co[i] = r_runreg(tmp)
    

In [30]:
co

array([[-3.11531354,  0.05525763, -1.50062562,  0.11046289],
       [-2.87042864,  0.03995998, -1.77695785,  0.12374685],
       [-3.18054303,  0.05729241, -1.37400344,  0.10788535],
       [-3.12414356,  0.06022228, -1.37404621,  0.09669057],
       [-3.04042576,  0.05064261, -1.4796746 ,  0.10399381],
       [-2.88113791,  0.03526393, -1.67356659,  0.10981958],
       [-2.96009378,  0.03996184, -1.59009974,  0.11106538],
       [-3.02956821,  0.05119976, -1.66040333,  0.11458737],
       [-3.02287287,  0.05309664, -1.70686082,  0.11644951],
       [-2.9164351 ,  0.04110651, -1.52907603,  0.10636362],
       [-2.97387818,  0.03221947, -1.66604255,  0.11618981],
       [-2.90769929,  0.04772813, -1.70383106,  0.11586265],
       [-3.01374717,  0.04483632, -1.652613  ,  0.11731724],
       [-3.14384826,  0.05215011, -1.49828283,  0.11095895],
       [-2.89990264,  0.04482563, -1.41460788,  0.09694476],
       [-2.89580686,  0.0411529 , -1.64936325,  0.10796648],
       [-2.94537869,  0.

In [31]:
print(co.mean(axis = 0))
print(co.std(axis = 0))

[-2.98461418  0.04413904 -1.58598083  0.11018178]
[ 0.09420597  0.00863835  0.12059695  0.00709176]


In [12]:
# # MC 
# B = 50 # replications
# MC1 = np.zeros((B, 4))
# # MC2 = np.zeros((B, 5))
# # MC3 = np.zeros((B, 5))
# # MC4 = np.zeros((B, 4))

# for i in range(B):
    
#     df = get_sim_data(T, k, gam, theta, n = 1000)
#     df = get_weights(df)
    
#     f = "Y ~ 1 + d1 + A + d3"
#     y, X = patsy.dmatrices(f, df.reset_index(), return_type = "dataframe")
#     mod = sm.Logit(y, X, missing="raise")
#     MC1[i] = mod.fit(disp=0).params.values

# #     f = "Y ~ 1 + d1 + A + d3 + L_100"
# #     y, X = patsy.dmatrices(f, df.reset_index(), return_type = "dataframe")
# #     mod = sm.Logit(y, X, missing="raise")
# #     MC2[i] = mod.fit().params.values

# #     f = "Y ~ 1 + d1 + A + d3 + Lav_100"
# #     y, X = patsy.dmatrices(f, df.reset_index(), return_type = "dataframe")
# #     mod = sm.Logit(y, X, missing="raise")
# #     MC3[i] = mod.fit().params.values

In [13]:
# print(MC1.mean(axis = 0))
# print(MC1.std(axis = 0))
# # print(MC2.mean(axis = 0))
# # print(MC2.std(axis = 0))
# # print(MC3.mean(axis = 0))
# # print(MC3.std(axis = 0))